In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-12-25 20:05:47--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
185.199.109.133, 185.199.110.133, 185.199.111.133, ...tent.com)... 
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
200 OKequest sent, awaiting response... 
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.3’

input.txt.3         100%[===================>]   1.06M  2.37MB/s    in 0.4s    

2024-12-25 20:05:48 (2.37 MB/s) - ‘input.txt.3’ saved [1115394/1115394]



In [3]:
with open('./input.txt', 'r') as f:
    text = f.read()
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [4]:
chars = sorted(list(set(text)))
print(''.join(chars))
vocab_sz = len(chars)
print(vocab_sz)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [5]:
stoi = {s:i for i,s in enumerate(chars)}
stoi

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'A': 13,
 'B': 14,
 'C': 15,
 'D': 16,
 'E': 17,
 'F': 18,
 'G': 19,
 'H': 20,
 'I': 21,
 'J': 22,
 'K': 23,
 'L': 24,
 'M': 25,
 'N': 26,
 'O': 27,
 'P': 28,
 'Q': 29,
 'R': 30,
 'S': 31,
 'T': 32,
 'U': 33,
 'V': 34,
 'W': 35,
 'X': 36,
 'Y': 37,
 'Z': 38,
 'a': 39,
 'b': 40,
 'c': 41,
 'd': 42,
 'e': 43,
 'f': 44,
 'g': 45,
 'h': 46,
 'i': 47,
 'j': 48,
 'k': 49,
 'l': 50,
 'm': 51,
 'n': 52,
 'o': 53,
 'p': 54,
 'q': 55,
 'r': 56,
 's': 57,
 't': 58,
 'u': 59,
 'v': 60,
 'w': 61,
 'x': 62,
 'y': 63,
 'z': 64}

In [6]:
itos = {i:s for i,s in enumerate(chars)}
itos

{0: '\n',
 1: ' ',
 2: '!',
 3: '$',
 4: '&',
 5: "'",
 6: ',',
 7: '-',
 8: '.',
 9: '3',
 10: ':',
 11: ';',
 12: '?',
 13: 'A',
 14: 'B',
 15: 'C',
 16: 'D',
 17: 'E',
 18: 'F',
 19: 'G',
 20: 'H',
 21: 'I',
 22: 'J',
 23: 'K',
 24: 'L',
 25: 'M',
 26: 'N',
 27: 'O',
 28: 'P',
 29: 'Q',
 30: 'R',
 31: 'S',
 32: 'T',
 33: 'U',
 34: 'V',
 35: 'W',
 36: 'X',
 37: 'Y',
 38: 'Z',
 39: 'a',
 40: 'b',
 41: 'c',
 42: 'd',
 43: 'e',
 44: 'f',
 45: 'g',
 46: 'h',
 47: 'i',
 48: 'j',
 49: 'k',
 50: 'l',
 51: 'm',
 52: 'n',
 53: 'o',
 54: 'p',
 55: 'q',
 56: 'r',
 57: 's',
 58: 't',
 59: 'u',
 60: 'v',
 61: 'w',
 62: 'x',
 63: 'y',
 64: 'z'}

In [7]:
encode = lambda x: [stoi[o] for o in x]
e = encode("hi there!")
e

[46, 47, 1, 58, 46, 43, 56, 43, 2]

In [8]:
decode = lambda x: ''.join([itos[o] for o in x])
d = decode(e)
''.join(d)

'hi there!'

In [9]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape)
print(data[:100])

torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [10]:
n = int(len(data) * 0.9)
trn_data = data[:n]
val_data = data[n:]
trn_data.shape, val_data.shape

(torch.Size([1003854]), torch.Size([111540]))

In [11]:
blk_sz = 8
trn_data[:blk_sz+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [12]:
x = trn_data[:blk_sz]
y = trn_data[1:blk_sz+1]
x,y

(tensor([18, 47, 56, 57, 58,  1, 15, 47]),
 tensor([47, 56, 57, 58,  1, 15, 47, 58]))

In [13]:
t = torch.randn((4, ))
t

tensor([ 2.1326, -0.0953,  0.3679, -0.5028])

In [14]:
for t in range(blk_sz):
    ctx = x[:t+1]
    tgt = y[t]
    print(f"Input: {ctx}; Target: {tgt}")

Input: tensor([18]); Target: 47
Input: tensor([18, 47]); Target: 56
Input: tensor([18, 47, 56]); Target: 57
Input: tensor([18, 47, 56, 57]); Target: 58
Input: tensor([18, 47, 56, 57, 58]); Target: 1
Input: tensor([18, 47, 56, 57, 58,  1]); Target: 15
Input: tensor([18, 47, 56, 57, 58,  1, 15]); Target: 47
Input: tensor([18, 47, 56, 57, 58,  1, 15, 47]); Target: 58


In [15]:
torch.manual_seed(1337)
batch_sz = 4
blk_sz = 8
ix = torch.randint(len(data) - blk_sz, (batch_sz, ))
ix

tensor([1078327,  453969,   41646,  671252])

In [16]:
i = ix[0]
data[i:i+blk_sz], decode(data[i:i+blk_sz].tolist())

(tensor([59, 52, 49, 47, 52, 42,  1, 40]), 'unkind b')

In [17]:
[data[i:i+blk_sz] for i in ix]

[tensor([59, 52, 49, 47, 52, 42,  1, 40]),
 tensor([53, 54, 43, 44, 59, 50,  1, 50]),
 tensor([27, 24, 33, 25, 26, 21, 13, 10]),
 tensor([47, 41, 43,  1, 53, 60, 43, 56])]

In [18]:
xbs = [data[i:i+blk_sz] for i in ix]
print([decode(x.tolist()) for x in xbs])

xbs = torch.stack(xbs)
xbs, xbs.shape # B, T

['unkind b', 'opeful l', 'OLUMNIA:', 'ice over']


(tensor([[59, 52, 49, 47, 52, 42,  1, 40],
         [53, 54, 43, 44, 59, 50,  1, 50],
         [27, 24, 33, 25, 26, 21, 13, 10],
         [47, 41, 43,  1, 53, 60, 43, 56]]),
 torch.Size([4, 8]))

In [19]:
ybs = [data[i+1:i+blk_sz+1] for i in ix]
print([decode(y.tolist()) for y in ybs])

ybs = torch.stack(ybs)
ybs, ybs.shape # B, T

['nkind br', 'peful la', 'LUMNIA:\n', 'ce overt']


(tensor([[52, 49, 47, 52, 42,  1, 40, 56],
         [54, 43, 44, 59, 50,  1, 50, 39],
         [24, 33, 25, 26, 21, 13, 10,  0],
         [41, 43,  1, 53, 60, 43, 56, 58]]),
 torch.Size([4, 8]))

In [20]:
torch.manual_seed(1337)
batch_sz = 4
blk_sz = 8

def get_batch(split):
    data = trn_data if split == 'train' else val_data
    ix = torch.randint(len(data) - blk_sz, (batch_sz, ))
    x = torch.stack([data[i  : i+blk_sz  ] for i in ix])
    y = torch.stack([data[i+1: i+blk_sz+1] for i in ix])
    return x,y

xb, yb = get_batch('train')
xb.shape, yb.shape

(torch.Size([4, 8]), torch.Size([4, 8]))

In [21]:
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.tkn_emb_table = nn.Embedding(vocab_sz, vocab_sz)

    def forward(self, idx, tgts=None):
        logits = self.tkn_emb_table(idx) # B, T, C
        # print("logits shape: ", logits.shape)
        if tgts is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            tgts = tgts.view(B*T)
            loss = F.cross_entropy(logits, tgts)
            
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx: B, T
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :] # B, C
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1) # B, 1
            idx = torch.cat((idx, idx_next), dim=1) # B, T+1
        return idx
            
m = BigramLanguageModel()
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


In [22]:
y_hats = m.generate(torch.zeros((4, 1), dtype=torch.long), 10)
print(decode(y_hats[0].tolist()))
print(decode(y_hats[1].tolist()))
print(decode(y_hats[2].tolist()))
print(decode(y_hats[3].tolist()))


Sr?qP-QWkt

S
vviYDEsx

,OMkDCV&oj

&Q:EJh$fRF


In [23]:
opt = torch.optim.AdamW(m.parameters(), lr=1e-3)

batch_sz = 32
for steps in range(10000):
    xb, yb = get_batch('train')
    
    logits, loss = m(xb, yb)
    opt.zero_grad(set_to_none=True)
    loss.backward()
    opt.step()
    
print(loss.item())

2.4730143547058105


In [24]:
print(decode(m.generate(torch.zeros((1, 1), dtype=torch.long), 300)[0].tolist()))


By y a!
Caril.n y, denjxhece w illd CHAL, mer thoun s's:Conchuntilalllevise sthat dy hangilyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte


In [25]:
model = BigramLanguageModel(vocab_sz)
opt = torch.optim.AdamW(model.parameters(), lr=1e-3)

TypeError: BigramLanguageModel.__init__() takes 1 positional argument but 2 were given

In [ ]:
eval_iters = 200
max_iters = 30000
eval_interval = 300

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            xb, yb = get_batch(split)
            logits, loss = model(xb, yb)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    
    return out

In [ ]:
for i in range(max_iters):

    if i % eval_interval == 0:
        losses = estimate_loss()
        print(f"step: {i} | train loss: {losses['train']:.4f} | val loss: {losses['val']:.4f}")
        
    xb, yb = get_batch('train')
    
    logits, loss = model(xb, yb)
    opt.zero_grad(set_to_none=True)
    loss.backward()
    opt.step()

In [ ]:
print(decode(model.generate(torch.zeros((1, 1), dtype=torch.long), 300)[0].tolist()))